# Deep Learning & Neural Networks

## Project 1A - non-linear classification

In this exericse, we will implement a simple neural network that classifies whether a 3D point is inside the unit sphere centered at 0. This neural network essentially *learns*, or tries to approximate, what the shape of a sphere is based on *limited data*.

The training data will be 100 arbitrary $\left(x, y, z\right)$ pairs together with labels indicating whether the points are in the sphere or not, i.e. whether for pair $(x, y, z)$:

$$ x^2 + y^2 + z^2 \le 1 $$

I'll explain a little later the format of the labels (this is somewhat important!)

### Setup

The first thing we'll do is load all the Python libraries we'll be using. See the comments below on what each library is for:

In [ ]:
# Load tensorflow
import tensorflow as tf
# Load numpy - adds MATLAB/Julia-style math to Python
import numpy as np
# Load matplotlib for plotting
import matplotlib.pyplot as plt
%matplotlib inline
# This is so that we can do some 3D plotting
from mpl_toolkits.mplot3d import Axes3D
# This is a convenient package for generating cartesian products
import itertools

We'll define a Python function that returns **True** if $(x, y, z)$ is contained inside a sphere and **False** otherwise. This is our *ground truth function*:

In [ ]:
def classify_inside_sphere(x, y, z):
    return x**2 + y**2 + z**2 < np.ones(x.shape)

The following code plots the sphere as a sanity check. We will be using the matplotlib library (loaded above) to generate a scatter plot.

In [ ]:
N_POINTS = 20

# generate some evenly spaced 3D points that are inside the sphere and we will plot them, so that what shows up is a sphere
points = filter(lambda arg: classify_inside_sphere(arg[0], arg[1], arg[2]),
                itertools.product(np.linspace(-1,1, N_POINTS), repeat=3))
xs = map(lambda x: x[0], points)
ys = map(lambda x: x[1], points)
zs = map(lambda x: x[2], points)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xs, ys, zs)

Just like in a lot of introductory machine learning classes, we will randomly generate training data.

For the labels we will use a 1-hot encoding. That is, each label is either the pair $(1, 0)$ if the corresponding point is outside the sphere and $(0, 1)$ if it's inside.

Think of this as something like data for a logistic regression problem with two classes. We will generalize this idea later (several classes, multinomial regression) with the MNIST project. 

In [ ]:
# Generate 100 random training data points
N_TRAIN = 100
SCALE = 1
xs = np.random.normal(scale=SCALE, size=N_TRAIN)
ys = np.random.normal(scale=SCALE, size=N_TRAIN)
zs = np.random.normal(scale=SCALE, size=N_TRAIN)
# Generate labels for the training points
inside_sphere = classify_inside_sphere(xs, ys, zs).astype(int)
true_labels = np.zeros((N_TRAIN, 2))
for i in range(N_TRAIN):
    true_labels[i, inside_sphere[i]] = 1

Now that we have our training data, we will finally start using *TensorFlow*! 

We need to describe, in Python code, a model for our neural network. In TensorFlow this model is called a **computation graph**, or simply a **graph**, and it tells us what mathematical functions should be applied to data.

If you're familiar with JuMP, you will see that TensorFlow is based on exactly the same idea of describing some mathematical model first and then running computations based on it!

### Entry points to our graph

The first thing to describe is what data will be used in computations (that are executed only **after** the graph is defined!).

For this we use the ``tf.placeholder`` function which returns us a *Tensor*, which is really just a multi-dimensional array. 

For our application we want a graph that calculates some loss function from training data, so our input tensors will be a batch of training points with their labels, so 2 tensors called: ``inputs`` and ``exp_output``.

When defining this placeholder Tensor, we should specify its dimensions but we want to give ourselves flexibility in what batch size (number of training points) is used. For this reason the dimensions of the ``inputs`` tensor will be $(?, 3)$ and for ``exp_output`` $(?, 2)$ where $?$ is a wildcard. That is, the batch size is unspecified and each training point consists of a 3D input vector and 2D expected binary output vector. In the Python code the wildcard is given by ``None`` reference.

In [ ]:
inputs = tf.placeholder(tf.float32, shape=[None, 3], name="inputs")
exp_output = tf.placeholder(tf.float32, [None,2])

Ok, so we have now defined our input and we will later be able to feed in data -- two numpy arrays with shape ``(?, 3)`` and ``(?, 2)``, for the training data and labels respectively. In the definition, we also supplied a ``name`` parameter: this is just to make debugging easier.

Let's now describe in our model what we will do with the ``inputs`` tensor -- we'll worry about ``exp_output`` tensor later.

In our neural network, the first layer is a fully connected one, so they operation we want to model is

$$Wx + b $$

where $W$ and $b$ are tensor *variables* for the weights, and biases, and $x$ stands for the input tensor. Let's define these variables and let's have 20 neurons for the hidden layer:

In [ ]:
N_HIDDEN = 20
hidden_W = tf.Variable(tf.random_normal([3, N_HIDDEN], stddev=0.3), name="hidden_weights")
hidden_b = tf.Variable(tf.zeros([N_HIDDEN]), name="hidden_bias")

We are now ready to describe the operation and using the TensorFlow functions we return and store a reference to the output of this fully connected layer (the result of $Wx + b$). The reference will be stored in the Python variable ``hidden_in`` (that is, the input to the hidden layer).

In [ ]:
hidden_in = tf.matmul(inputs, hidden_W) + hidden_b
hidden_out = tf.sigmoid(hidden_in)

We have immediately passed the ``hidden_in`` tensor to the sigmoid activation function. This adds a node to our graph which represents the element-wise sigmoid function, and we call the output tensor ``hidden_out``.

**Question:** What is the dimension of the ``hidden_out`` tensor? Why?

**Answer:** $(?, N)$ where $N$ is the number of hidden layer neurons. Recall that we are multiplying a $(?, 3)$ tensor with $W$, the latter having dimensions $(3, N)$.

Note that we can easily check his using TensorFlow:

In [ ]:
hidden_out.get_shape()

Now let's add another fully connected layer. This time the output will have dimension $(?, 2)$ -- use the previously mentioned function to check this!

In [ ]:
output_W = tf.Variable(tf.random_normal([N_HIDDEN, 2], stddev=0.3), name="output_weights")
output_b = tf.Variable(tf.zeros([2]), name="output_bias")
output = tf.matmul(hidden_out, output_W) + output_b
output.get_shape()

Now the final thing our neural network needs to do is give us a 2-dimensional vector as output whose entries are non-negative and sum to 1 (probability distribution). We do this using the softmax function.

Recall the softmax function is defined as:

$$\sigma(y)_k = \frac{e^{y_k}}{\sum_j e^{y_j}}$$

In [ ]:
net_output = tf.nn.softmax(output)

Since this like the sigmoid function is elementwise, the ``net_output`` tensor has shape $(?,2)$. Again - please check this for yourself!

Voila! We now have the complete definition of our neural network, which has as an input layer, the tensor ``inputs``, as well as 1 hidden layer. When the network is instantiated, all its weights and bias parameters will be initialized to random values. However, the point is we want to train the parameters using the data, so in the following code we will define the operations that are necessary for this.

But before we go ahead with that, let's see how we can get some output from our neural network. We need to do this with a ``Session`` object.

In [ ]:
session = tf.Session()

Now that we have our ``session`` let's first do the random initialization of parameters. First we create the operation:

In [ ]:
init = tf.initialize_all_variables()

Then we get the ``session`` object to run this initialization:

In [ ]:
session.run(init)

Our network is now initialized, and since it's not been trained at all, will output garbage. But let's just see that it's the right format.

To run a forward pass through the newtwork, we need to *run* the operation corresponding to ``net_output`` using some Numpy array that we shall feed in:

In [ ]:
# First mash the inputs to answer
xs_tensor = np.vstack((xs, ys, zs))

# Then feed into network by running the session
session.run(net_output, feed_dict={inputs: xs_tensor.T})

You can convince yourself, as an exercise, that this output is a bad classifier of points in a sphere. 

Let's now train our network, to do this we need to define a loss function. 

In [ ]:
cross_entropy = tf.reduce_mean(-exp_output*tf.log(net_output))

We have essentially extended our computation graph with yet more operations on tensors and this time notice how we finally got to use our ``exp_output`` placeholder. The final output, referenced by ``cross_entropy``, represents a scalar value that we want to minimize.

The minimization will be done using Stochastic Gradient Descent (SGD). This is another step in our code (but behind the scenes TensorFlow adds all the low level operations that occur during backpropogation, building the graph that computes gradients etc.)

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(0.5)
train_step = optimizer.minimize(cross_entropy)

The 0.5 passed into the ``tf.train.GradientDescentOptimizer`` constructor is the learning rate for SGD. Note that the reference for the minimization step is the Python variable ``train_step``.

### Training and running the network

We have finished describing the network in our code and TensorFlow has "built" a training algorithm from all this.

We run the gradient step 10,000 times by **running** the train step operation and feeding in appropriate Numpy arrays for both the ``inputs`` and ``exp_output`` tensors.

In [ ]:
# Before running the algorithm, we need to mash our training data into
# a tensor using numpy
xs_tensor = np.vstack((xs, ys, zs))

# Run some gradient steps and plot the cross-entropy loss over time
N_STEPS = 10000
errors = []
for i in xrange(N_STEPS):
    train_error, _ = session.run((cross_entropy, train_step), 
                              feed_dict={inputs : xs_tensor.T, exp_output: true_labels})
    if i%1000==0: print i, train_error
    errors.append(train_error)
plt.plot(range(N_STEPS), errors, 'b-')

The training is now complete and we will want to measure the accuracy of our network on training data. We do this by building another TensorFlow operation!

In [ ]:
# Now we are going to create an operation for measuring the network's accuracy, 
# that is the percentage of samples that the network classifies correctly.
# Think of this as an alternative loss function, which is **not** the one being minimized
correct_points = tf.cast(tf.equal(tf.argmax(exp_output, 1), tf.argmax(net_output, 1)), tf.float32)
accuracy = tf.reduce_mean(correct_points)

We can run the new operation using the exact same Numpy arrays as inputs like before:

In [ ]:
# We know the cross-entropy is low -- what about the accuracy?
# By calling the accuracy operation inside the session and feeding in the training data we'll get 
session.run(accuracy, feed_dict={inputs : xs_tensor.T, exp_output : true_labels})

We know from Machine Learning principles that just because our network fits the training data well, doesn't always mean it generalizes. We are now going to check what actually happens when we test "edge cases", points close to the surface of the sphere which are more likely to be mis-classified.

In [ ]:
# We can see that the accuracy is close to 100% on the training data (good!)
# However, did we just overfit or did our classifier accurately learn the shape of the sphere?

# We're going to generate some test data around the surface of one of the sphere's quadrants
# The idea is we want to essentially test these difficult 'corner cases'
thetas = np.arange(0.2, np.pi/2, 0.02)
phis = np.arange(0.2, np.pi/2, 0.02)
xs_test = []
ys_test = []
zs_test = []
for phi in phis:
    z = np.cos(phi)
    r = np.sin(phi)
    for theta in thetas:
        y = r*np.cos(theta)
        x = r*np.sin(theta)
        xs_test.append(x)
        ys_test.append(y)
        zs_test.append(z)
xs_test = np.asarray(xs_test)
ys_test = np.asarray(ys_test)
zs_test = np.asarray(zs_test)
xs_test += np.random.normal(scale=0.5, size=len(xs_test))

# Let's try to trick our classifier by using points outside the sphere but close to its surface
bad_indices = filter(lambda i: not classify_inside_sphere(xs_test[i], ys_test[i], zs_test[i]), range(len(xs_test)))
xs_test = xs_test[bad_indices]
ys_test = ys_test[bad_indices]
zs_test = zs_test[bad_indices]
xs_test_tensor = np.vstack((xs_test, ys_test, zs_test))

The above code generated the edge cases: points close to the surface but outside of the sphere. Now we define yet another TensorFlow operation to count and average out the number of mistakes. We immediately run it in our ``session`` to get the answer

In [ ]:
# Finally we can just get the accuracy by subtracting from 1 the fraction of false positives 
# (remember there are no false negatives)
classifications = tf.argmax(net_output, 1)
1 - np.mean(session.run(classifications, feed_dict={inputs : xs_test_tensor.T}))

So unfortunately, our network doesn't perform that well out-of-sample. Using the ``classifications`` operations again (which simply takes argmax of the softmax output) we can plot the boundary of our classifier

In [ ]:
# Let's plot what the boundary of our non-linear classifier looks like
points_array = np.asarray([elem for elem in itertools.product(np.linspace(-1,1, N_POINTS), repeat=3)])
classified = session.run(classifications, feed_dict={inputs: points_array}).astype(bool)
points_to_plot = points_array[filter(lambda i: classified[i], range(len(points_array)))]
xs = map(lambda x: x[0], points_to_plot)
ys = map(lambda x: x[1], points_to_plot)
zs = map(lambda x: x[2], points_to_plot)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xs, ys, zs)

Looks somewhat like a deformed sphere, but this is what happens when you have a complex model like a neural network and not enough training points.

## Project 1B. Exercise: non-linear regression

In this exercise, you will implement a neural network that learns the Rosenbrock function, defined as:

$$ f(x,y) = (a-x)^2 + b(y -x^2)^2 $$

The architecture should almost the same as the previous network, only differences are:

  * 2-dimensional input layer
  * 1-dimensional linear output layer - so **no** softmax!
  * Square loss function used for training $\frac{1}{2}(f(x) - y)^2$

But remember to use (like before):
  * 1 hidden layer with sigmoid activation function
  * 20 neurons in the hidden layer
  
The only thing I haven't shown yet is that ``tf.square`` is the function for implementing the elementwise squaring operation in TensorFlow.

Once you've finished the exercise check out the slides to see what extensions you can try for this and the previous project.

In [ ]:
# We'll use the classic Rosenbrock function
def rosenbrock(x,y):
    a, b = 1.0, 100.0
    return (a - x) ** 2 + b * (y - x**2) ** 2
# Lets plot our function
x = np.outer(np.linspace(-2,+2,100), np.ones(100))
y = np.outer(np.ones(100), np.linspace(-1,+4,100))
z = rosenbrock(x, y)
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(x, y, z, cmap=plt.cm.jet)

Now that we've defined the Rosenbrock function, let's get some training data to fit our network to:

In [ ]:
# Lets generate some training data
N_TRAIN = 500
# Sample x from -2 to +2
x_train = np.random.rand(N_TRAIN) * 4 - 2
# Sample y from -1 to 4
y_train = np.random.rand(N_TRAIN) * 5 - 1
# Calculate z
z_train = rosenbrock(x_train, y_train)
# Plot the training set
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(x, y, z, cmap=plt.cm.jet, alpha=0.2)
ax.scatter(x_train, y_train, z_train, c="white", s=10)

**Exercise:** build the network! Make sure you call the input ``net_input`` and labels ``exp_output`` and the training operation, ``train_step`` (so that you can run the code that follows after this cell). Basically, fill in the parts marked below with comments:

In [ ]:
N_HIDDEN = 20

net_input = tf.placeholder(tf.float32, [None,2])
# TODO: fill in these parts by defining the computation graph...
# Remember N_HIDDEN = 20 hidden neurons, sigmoid activation and 1-D real-valued output layer

exp_output = tf.placeholder(tf.float32, [None,1])
# TODO: define the square loss function and call it 'mse'
# hint: tf.square(...) might be useful

# You will use this optimizer
optimizer = tf.train.GradientDescentOptimizer(0.0005)
train_step = optimizer.minimize(mse)

# Usual boilerplate code to create a session and initialize the variables you just defined
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

The below cells will train and test your network. If your solution is correct, the accuracy should be close to 100% on training data.

In [ ]:
# Run the network to get the output on our training data
# Before we do, we need to mash our training data into
# a tensor
train_xy_tensor = np.vstack((x_train, y_train)).T
train_z_tensor = np.reshape(z_train, (N_TRAIN,1))
initial_z = sess.run(net_output, feed_dict={
                        net_input: train_xy_tensor,
                        exp_output: train_z_tensor})
initial_z = np.reshape(initial_z, len(initial_z))
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.scatter(x_train, y_train, z_train, c="white", s=10)
ax.scatter(x_train, y_train, initial_z, c="purple", s=10)

Now you've built the function approximator, you can look at what it thinks the function is

In [ ]:
x_test_mat = np.outer(np.linspace(-2,+2,100), np.ones(100))
y_test_mat = np.outer(np.ones(100), np.linspace(-1,+4,100))
x_test = np.reshape(x_test_mat, (100*100,1))
y_test = np.reshape(y_test_mat, (100*100,1))
test_xy_tensor = np.hstack((x_test, y_test))
net_z_test = sess.run(net_output, feed_dict={net_input: test_xy_tensor})
z_test_mat = np.reshape(net_z_test, (100,100))
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(x_test_mat, y_test_mat, z_test_mat, cmap=plt.cm.jet)

You can also check out its accuracy

In [ ]:
x = np.outer(np.linspace(-2,+2,100), np.ones(100))
y = np.outer(np.ones(100), np.linspace(-1,+4,100))
z = rosenbrock(x, y)
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(x, y, z, cmap=plt.cm.jet)
print np.mean((z - z_test_mat)**2)